#### Part 1 - get dataframe of postalcodes per neighbourhoods

In [1]:
import pandas as pd
#!conda install beautifulsoup4
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # wiki page containing a table of Toronto neighborhoods
r = requests.get(url) 
soup = BeautifulSoup(r.content,"html5lib")  # html content of the url
tables = soup.find('table', class_ = 'wikitable sortable') # get table
rows = tables.find_all('tr') # get table rows

postcodes = []
boroughs = []
neighborhoods = []

for row in rows:
    col = row.find_all('td')
    try:
        if col[1].text != 'Not assigned': # skip rows with not assigned
            postcodes.append(col[0].text)
            boroughs.append(col[1].text)
            nbds_tmp = col[2].text.split('\n')[0] # remove new line tag
            if nbds_tmp == 'Not assigned': # if neighborhood isnt assigned, use borough instead
                neighborhoods.append(col[1].text)
            else:
                neighborhoods.append(nbds_tmp)    
    except: # first row
        pass
    

d = {'Postcode': postcodes, 'Borough': boroughs ,'Neighborhood': neighborhoods}
tor_nbds = pd.DataFrame(data=d)
tor_nbds = tor_nbds.groupby(['Postcode','Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()
tor_nbds.head()
print('Number of rows: ', tor_nbds.shape[0])
print('Number of cols: ', tor_nbds.shape[1])

Number of rows:  103
Number of cols:  3


####  Part 2 - Get coordinates per postal code
I used pgeocode instead of geocoder because the latter didn't work. 

In [2]:
#!pip install pgeocode
import pgeocode

In [3]:
pcodes = tor_nbds['Postcode'].values # get a list of postcodes
nomi = pgeocode.Nominatim('ca') # Get info on all neighborhoods in Canada
lat = []
lon = []

for pcode in pcodes: # loop over each postcode and get the coordinates
    pcode_info = nomi.query_postal_code(pcode)
    lat.append(pcode_info.latitude)
    lon.append(pcode_info.longitude)

tor_nbds['Latitude'] = lat
tor_nbds['Longitude'] = lon

tor_nbds.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
